[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/education-toolkit/blob/main/02_ml-demos-with-gradio.ipynb)


Transformers Pipeline Demo Notebook

💡 **Welcome!**

This notebook is a self-contained way to start using widely known Open Source technologies (`transformers`, `gradio`, etc). The core use case is building an application to search Wikipedia.  This notebook can be found at [https://bit.ly/raj_askwiki](https://bit.ly/raj_askwiki), the accompanying slides are at [****](https://github.com/rajshah4/huggingface-demos/tree/main/FinBERT).


In [ ]:
!pip install --quiet transformers
!pip install --quiet gradio
!pip install --quiet sentence-transformers
!pip install --quiet datasets
import os
import gzip
import json

# Tutorial: Using Pretrained Models and Building Demos with Gradio ⚡ & Hugging Face 🤗 

**Learning goals:** The goal of this tutorial is to learn How To

1. Use pre-trained models from the transformers library and that are available on the Hugging Face Hub
2. Extract information from Pre-Trained Models
3. Doing a similarity search
4. Building a web demo, [Ask Wiki](https://huggingface.co/spaces/rajistics/Ask-Wiki)

**Duration**: 60 minutes

**Prerequisites:** Knowledge of Python and basic familiarity with machine learning 

**Author**: [Rajiv Shah](https://twitter.com/rajistics) (feel free to ping me with any questions about this tutorial) 

All of these steps can be done for free! All you need is an Internet browser and a place where you can write Python 👩‍💻

# Let's Start by Exploring Pre-trained Models at Hugging Face Hub
[Hugging Face Hub](https://hf.co) 

**Voice:** Automatic Speech Recognition [Facebook's Wav2Vec2](https://huggingface.co/facebook/wav2vec2-base-960h)

**Image:** Object Detection [DETR End-to-End Object Detection model with ResNet-50 backbone](https://huggingface.co/facebook/detr-resnet-50)

## Let's Run Pretrained Models for Predictions

The [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) supports many common tasks out-of-the-box:

**Text**:
* Sentiment analysis: classify the polarity of a given text.
* Text generation (in English): generate text from a given input.
* Name entity recognition (NER): label each word with the entity it represents (person, date, location, etc.).
* Question answering: extract the answer from the context, given some context and a question.
* Fill-mask: fill in the blank given a text with masked words.
* Summarization: generate a summary of a long sequence of text or document.
* Translation: translate text into another language.
* Feature extraction: create a tensor representation of the text.

**Image**:
* Image classification: classify an image.
* Image segmentation: classify every pixel in an image.
* Object detection: detect objects within an image.

**Audio**:
* Audio classification: assign a label to a given segment of audio.
* Automatic speech recognition (ASR): transcribe audio data into text.

<Tip>

For more details about the [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) and associated tasks, refer to the documentation [here](https://huggingface.co/docs/transformers/main/en/./main_classes/pipelines).


###Text

In [ ]:
#Sentiment Analysis
from transformers import pipeline
sent_classifier = pipeline("sentiment-analysis")
sent_classifier("I was pretty happy with the sneakers")

In [ ]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [ ]:
summarizer = pipeline("summarization")

In [ ]:
summarizer(ARTICLE, max_length=130, min_length=30, do_sample=True)

[{'summary_text': ' Liana Barrientos has been married 10 times, nine of them between 1999 and 2002 . She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say . She pleaded not guilty at State Supreme Court in the Bronx on Friday .'}]

In [ ]:
#Zero Shot Learning
zero_shot = pipeline('zero-shot-classification')
zero_shot(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

###Image

In [ ]:
vision_classifier = pipeline(task="image-classification")
result = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
print("\n".join([f"Class {d['label']} with score {round(d['score'], 4)}" for d in result]))

## Finetuned FinBERT for Forward Looking Statements

[FinBERT-FLS](https://huggingface.co/yiyanghkust/finbert-fls) is model developed for identifying Forward-looking statements (FLS). These statements inform investors of managers’ beliefs and opinions about firm's future events or results. Identifying forward-looking statements from corporate reports can assist investors in financial analysis. FinBERT-FLS is a FinBERT model fine-tuned on 3,500 manually annotated sentences from Management Discussion and Analysis section of annual reports of Russell 3000 firms.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp('In the past, the age of our fleet to enhance availability and reliability due to reduced downtime for repairs.')
print(results) 

## Inference Pipelines from Hugging Face

For production use, you can use the [inference API](https://huggingface.co/inference-api) to get predictions via simple API calls.  To get the snippet, just go here
![](https://i.ibb.co/P9yyTHg/Screen-Shot-2022-07-01-at-10-30-20-AM.png)

In [ ]:
# Example Snippet
import requests

API_URL = "https://api-inference.huggingface.co/models/yiyanghkust/finbert-fls"
headers = {"Authorization": "Bearer {API_TOKEN}"}   ###Add your API Key here after Bearer

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "In the past, the age of our fleet to enhance availability and reliability due to reduced downtime for repairs.",
})

# Extracting Information from Pre-trained Models

[Embedding Model Used in the Presentation](https://huggingface.co/obrizum/all-MiniLM-L6-v2)

In [ ]:
sentences = ["This is an example sentence", "Each sentence is converted"]

from sentence_transformers import SentenceTransformer, util, CrossEncoder

model = SentenceTransformer('obrizum/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

# Similarity Search


A great visualization tool for embeddings is the [Tensorflow Projector](https://projector.tensorflow.org/)

[Similarity Model Used in the Presentation](https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1)

Let's use the concept of embeddings now to match a query with the best related passage in some documents. This demo will use an extract of Wikipedia as the passages to search.

In [ ]:
## Get Wikipedia data
wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

In [ ]:
passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        #Add all paragraphs
        #passages.extend(data['paragraphs'])

        #Only add the first paragraph
        passages.append(data['paragraphs'][0])

print("Passages:", len(passages))


Passages: 169597


In [ ]:
data

{'id': '798870',
 'paragraphs': ['The Seminole bat ("Lasiurus seminolus") is a type of bat in the family Vespertilionidae.',
  'The Seminole bat is often confused with the red bat. The Seminole bat has a mahogany color with a frosted look because to white tipped dorsal hairs. They weigh around 12 grams. Females are larger than males.',
  'The Seminole bat is found in the Southeastern United States. This includes Louisiana, Georgia, Alabama, Mississippi, South Carolina and parts of Texas, Tennessee, Arkansas and North Carolina. It has also been seen as far as Mexico. It is a migratory species. In the winter, it lives along the Gulf Coast, North and South Carolina, and southern Arkansas. In the summer, they migrate as far north as Missouri and Kentucky.',
  'It prefers to live in forested areas. In winter, they are found to use leaf litter and Spanish moss as insulation in their roost sites.',
  'Seminole bats are insectivores. They eat large amounts of Hymenoptera (ants, bees and wasps)

In [ ]:
passages[0:5]

['Ted Cassidy (July 31, 1932 - January 16, 1979) was an American actor. He was best known for his roles as Lurch and Thing on "The Addams Family".',
 'Aileen Carol Wuornos Pralle (born Aileen Carol Pittman; February 29, 1956\xa0– October 9, 2002) was an American serial killer. She was born in Rochester, Michigan. She confessed to killing six men in Florida and was executed in Florida State Prison by lethal injection for the murders. Wuornos said that the men she killed had raped her or tried to rape her while she was working as a prostitute.',
 "A crater is a round dent on a planet. They are usually shaped like a circle or an oval. They are usually made by something like a meteor hitting the surface of a planet. Underground activity such as volcanoes or explosions can also cause them but it's not as likely.",
 'Store has several meanings:',
 'Chinese New Year, known in China as the SpringFestival and in Singapore as the LunarNewYear, is a holiday on and around the new moon on the first

In [ ]:
#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder


In [ ]:

# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
query="What is the capital of the United States?"

In [ ]:
# Encode the query using the bi-encoder and find potentially relevant passages
question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
question_embedding = question_embedding.cuda()
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
hits = hits[0]  # Get the hits for the first query

In [ ]:
print("\n-------------------------\n")
print("Top-3 Bi-Encoder Retrieval hits")
hits = sorted(hits, key=lambda x: x['score'], reverse=True)
for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))


For a more advanced similarity search, let's add a cross encoder for reranking. If you want to see a more thorough comparison against lexicon searching, check out [https://bit.ly/raj_semantic/](https://bit.ly/raj_semantic/)


In [ ]:
##Go add a Cross Encoder for Reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

##### Re-Ranking #####
# Now, score all retrieved passages with the cross_encoder
cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
cross_scores = cross_encoder.predict(cross_inp)

# Sort results by the cross-encoder scores
for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]

# Output of top-5 hits from re-ranker
print("\n-------------------------\n")
print("Top-3 Cross-Encoder Re-ranker hits")
hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))

# Let's Build a Demo

**Demos** of machine learning models are an increasingly important part of machine learning. Demos allow:

* model developers to easily **present** their work to a wide audience
* increase **reproducibility** of machine learning research
* diverse users to more easily **identify and debug** failure points of models


As a quick example of what we would like to build, check out the [Keras Org on Hugging Face](https://huggingface.co/keras-io), which includes a description card and a collection of Models and Spaces built by Keras community. Any Space can be opened in your browser and you can use the model immediately, as shown here: 

![](https://i.ibb.co/7y6DGjB/ezgif-5-cc52b7e590.gif)




## 1. Build Quick ML Demos in Python Using the Gradio Library

`gradio` is a handy Python library that lets you build web demos simply by specifying the list of input and output **components** expected by your machine learning model. 

For more detail [see the docs](https://gradio.app/docs/)

In addition to the input and output types, Gradio expects a third parameter, which is the prediction function itself. This parameter can be ***any* regular Python function** that takes in parameter(s) corresponding to the input component(s) and returns value(s) corresponding to the output component(s)

Enough words. Let's see some code!

In [ ]:
#from sentence_transformers import SentenceTransformer, CrossEncoder, util
#import torch
##import pickle
#import pandas as pd
#import gradio as gr

In [ ]:
import gradio as gr

def search(query,top_k=100):
    ans=[]
    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for idx, hit in enumerate(hits[0:3]):
        ans.append(passages[hit['corpus_id']])
    return ans[0],ans[1],ans[2]

iface = gr.Interface(fn=search, 
                     inputs=gr.Textbox(label="Query"), 
                     outputs=[gr.Textbox(label="Answer 1"),gr.Textbox(label="Answer 2"),gr.Textbox(label="Answer 3")])

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://58549.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


## 2. Host the Demo (for free) on Hugging Face Spaces

Once you made a Gradio demo, you can host it permanently on Hugging Spaces very easily:

Here are the steps to that (shown in the GIF below):

A. First, create a Hugging Face account if you do not already have one, by visiting https://huggingface.co/ and clicking "Sign Up"

B. Once you are logged in, click on your profile picture and then click on "New Space" underneath it to get to this page: https://huggingface.co/new-space

C. Give your Space a name and a license. Select "Gradio" as the Space SDK, and then choose "Public" if you are fine with everyone accessing your Space and the underlying code

D. Then you will find a page that provides you instructions on how to upload your files into the Git repository for that Space. You may also need to add a `requirements.txt` file to specify any Python package dependencies.

E. Once you have pushed your files, that's it! Spaces will automatically build your Gradio demo allowing you to share it with anyone, anywhere!

![GIF](https://huggingface.co/blog/assets/28_gradio-spaces/spaces-demo-finalized.gif)





Check out [Ask Wiki](https://huggingface.co/spaces/rajistics/Ask-Wiki) by rajistics 
